<a href="https://colab.research.google.com/github/RagilAkbari/BoardGame-Analysis/blob/main/jadi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:

import tensorflow
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot
from keras.callbacks import EarlyStopping
import time

In [7]:
from google.colab import files #upload data
uploaded = files.upload()

Saving bgg_db2.csv to bgg_db2 (1).csv


In [8]:
import io
data = pd.read_csv('bgg_db2.csv')

In [9]:
data.head()

,game_id,names,category,min_players,max_players,min_time,max_time,avg_time,year,avg_rating,geek_rating,num_votes,rank,owned
0,1,Die Macher,"Economic, Negotiation, Political",3,5,240,240,240,1986,7.62437,7.15383,5001,255.0,6615
1,2,Dragonmaster,"Card Game, Fantasy",3,4,30,30,30,1981,6.61497,5.80921,540,3444.0,1224
2,3,Samurai,"Abstract Strategy, Medieval",2,4,30,60,60,1998,7.44113,7.24689,14155,199.0,14500
3,4,Tal der Könige,Ancient,2,4,60,60,60,1992,6.60429,5.70264,331,4530.0,607
4,5,Acquire,Economic,2,6,90,90,90,1964,7.34541,7.16840,17672,247.0,22352


Menghilangkan Data yang "duplicate", null / missing value

In [11]:
data.count()

game_id        116455
names          116455
category       114405
min_players    116455
max_players    116455
min_time       116455
max_time       116455
avg_time       116455
year           116455
avg_rating     116455
geek_rating    116455
num_votes      116455
rank            18789
owned          116455
dtype: int64

In [12]:
data = data.drop_duplicates() #drop duplikasi

In [13]:
data.count()

game_id        116455
names          116455
category       114405
min_players    116455
max_players    116455
min_time       116455
max_time       116455
avg_time       116455
year           116455
avg_rating     116455
geek_rating    116455
num_votes      116455
rank            18789
owned          116455
dtype: int64

In [14]:
data = data.dropna() #drop nilai Null

In [15]:
data.count()

game_id        18580
names          18580
category       18580
min_players    18580
max_players    18580
min_time       18580
max_time       18580
avg_time       18580
year           18580
avg_rating     18580
geek_rating    18580
num_votes      18580
rank           18580
owned          18580
dtype: int64

Mengubah data Average Rating menjadi 2 macam saja

In [16]:
#nilai rating dari 0-10, rating < 5 == 0, sisa 1. Sebagai pembatas rating tinggi dan rendah
data['avg_rating'] = (data['avg_rating'] >= 5).astype(int)
data.head() 

,game_id,names,category,min_players,max_players,min_time,max_time,avg_time,year,avg_rating,geek_rating,num_votes,rank,owned
0,1,Die Macher,"Economic, Negotiation, Political",3,5,240,240,240,1986,1,7.15383,5001,255.0,6615
1,2,Dragonmaster,"Card Game, Fantasy",3,4,30,30,30,1981,1,5.80921,540,3444.0,1224
2,3,Samurai,"Abstract Strategy, Medieval",2,4,30,60,60,1998,1,7.24689,14155,199.0,14500
3,4,Tal der Könige,Ancient,2,4,60,60,60,1992,1,5.70264,331,4530.0,607
4,5,Acquire,Economic,2,6,90,90,90,1964,1,7.16840,17672,247.0,22352


In [17]:
feature_cols = ['min_players','max_players','year','geek_rating','num_votes','owned'] #Fitur2 yg digunakan
X = data[feature_cols]
y = data.avg_rating #target

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

In [21]:
X_train #cek data train

,min_players,max_players,year,geek_rating,num_votes,owned
0,3,5,1986,7.15383,5001,6615
1,3,4,1981,5.80921,540,1224
2,2,4,1998,7.24689,14155,14500
3,2,4,1992,5.70264,331,607
4,2,6,1964,7.16840,17672,22352
...,...,...,...,...,...,...
71076,1,4,2014,5.52734,66,198
71079,1,1,2014,5.59018,154,400
71081,3,16,2014,5.51934,34,82
71084,2,2,2015,5.66756,206,613


In [22]:
X_test #cek data test

,min_players,max_players,year,geek_rating,num_votes,owned
71092,3,10,2014,5.58744,277,602
71094,2,5,2014,5.64748,378,1327
71095,2,2,2016,5.58204,89,236
71096,2,2,2014,5.60465,99,349
71098,2,2,2015,6.26338,650,2085
...,...,...,...,...,...,...
113339,2,6,2020,5.60093,109,185
114664,2,5,2020,5.57559,47,187
114802,1,4,2020,5.53111,33,27
114865,3,7,2020,5.71095,123,434


In [23]:
#model Sequential dengan 3 layer
model=keras.models.Sequential([
    keras.layers.Dense(64, input_dim = X_train.shape[1], activation='tanh'),  
    keras.layers.Dropout(0.3),
    
    keras.layers.Dense(units=32, activation='tanh'), 
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(units=1, activation="sigmoid"),
],name="Initial_model",)
model.summary()

Model: "Initial_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                448       
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,561
Trainable params: 2,561
Non-trainable params: 0
_________________________________________________________________


In [24]:
#up learning rate paling kecil
learning_rate = 0.01
optimizer = keras.optimizers.Adam(lr=learning_rate)

In [25]:
model.compile(optimizer=optimizer,
            loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
#fungsi class untuk menghentikan training ketika epoch tidak ada perkembangan
early_stopping_monitor = EarlyStopping( 
    monitor='val_loss',
    min_delta=0,
    patience=100,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

In [32]:
#proses training
start = time.clock() 
history = model.fit(X_train, y_train, epochs=1000, batch_size=512, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping_monitor])
end = time.clock()

Epoch 1/1000
26/26 [==============================] - 0s 7ms/step - loss: 0.2927 - accuracy: 0.9098 - val_loss: 0.1360 - val_accuracy: 0.9824
Epoch 2/1000
26/26 [==============================] - 0s 4ms/step - loss: 0.2947 - accuracy: 0.9098 - val_loss: 0.1172 - val_accuracy: 0.9824
Epoch 3/1000
26/26 [==============================] - 0s 5ms/step - loss: 0.2969 - accuracy: 0.9098 - val_loss: 0.1567 - val_accuracy: 0.9824
Epoch 4/1000
26/26 [==============================] - 0s 4ms/step - loss: 0.2922 - accuracy: 0.9098 - val_loss: 0.1253 - val_accuracy: 0.9824
Epoch 5/1000
26/26 [==============================] - 0s 4ms/step - loss: 0.2923 - accuracy: 0.9098 - val_loss: 0.1224 - val_accuracy: 0.9824
Epoch 6/1000
26/26 [==============================] - 0s 4ms/step - loss: 0.2923 - accuracy: 0.9098 - val_loss: 0.1229 - val_accuracy: 0.9824
Epoch 7/1000
26/26 [==============================] - 0s 4ms/step - loss: 0.2923 - accuracy: 0.9098 - val_loss: 0.1254 - val_accuracy: 0.9824
Epoch 

In [33]:
print("Train time: {} ".format(end-start)) #cek waktu run train

Train time: 25.597061999999994 


In [34]:
#cek akurasi antara X_test dan y_test
model.evaluate(x=X_test,y=y_test) 

175/175 [==============================] - 0s 878us/step - loss: 0.1082 - accuracy: 0.9824


[0.10824409872293472, 0.9824183583259583]